# Partie A : 
## 1) Description de l’unité statistique d’observation :

C'est le produit cultivé pour chaque année et chaque pays 
  
  fnid : identifiant géographique unique (FEWS NET)
  admin_1 : unité administrative de premier niveau
  admin_2 : unité administrative de second niveau
  product : culture agricole (ex. maïs, sorgho, millet)
  season_name : saison de culture
## 2) Explication de la différence conceptuelle entre area, production et yield et leur relation

La variable ‘’area’’ représente la surface cultivée en Ha et la variable ‘’production’’ la quantité agricole produite en tonnes. Le rapport entre ces deux donne le rendement ‘’yield’’. On a la formule suivante : yield =  production / area

### NB : les autres réponses sont dans le script R

# Analyse spatiale pertinente de la base de données HarvestStat Africa :
### cas du senegal 

In [56]:
# ==========================================
# ANALYSE SPATIALE AGRICOLE – SENEGAL
# ==========================================

import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from libpysal import weights             # pour les matrices de voisinage
from esda.moran import Moran, Moran_Local # indices de Moran et LISA

# ------------------------------------------
# 1. DOSSIER OUTPUT
# ------------------------------------------
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------
# 2. CHARGEMENT DES DONNEES
# ------------------------------------------
df = pd.read_csv("data/harveststat.csv")
gdf = gpd.read_file("data/harveststat.gpkg")

# ------------------------------------------
# 3. FILTRAGE SENEGAL + NETTOYAGE
# ------------------------------------------
df["fnid"] = df["fnid"].astype(str)
gdf["FNID"] = gdf["FNID"].astype(str)

df = df[
    (df["country"] == "Senegal") &
    (df["qc_flag"] == 0)
].copy()

# ------------------------------------------
# 4. AGRÉGATION SPATIALE (ADMIN_2)
# ------------------------------------------
agg_admin2 = (
    df.groupby("fnid")
      .agg(
          mean_yield=("yield", "mean"),
          total_production=("production", "sum"),
          mean_area=("area", "mean")
      )
      .reset_index()
)

# ------------------------------------------
# 5. JOINTURE SPATIALE
# ------------------------------------------
gdf_sn = gdf.merge(
    agg_admin2,
    left_on="FNID",
    right_on="fnid",
    how="inner"
)

# ------------------------------------------
# 6. CARTE DES RENDEMENTS
# ------------------------------------------
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
gdf_sn.plot(
    column="mean_yield",
    cmap="YlGn",
    legend=True,
    ax=ax
)
ax.set_title("Rendement agricole moyen au Sénégal (t/ha)")
ax.axis("off")
plt.savefig(f"{OUTPUT_DIR}/senegal_yield_map.png", dpi=300)
plt.close()

# ------------------------------------------
# 7. ZONES LES PLUS FAIBLES
# ------------------------------------------
low_yield = (
    gdf_sn[["ADMIN2", "mean_yield"]]
    .sort_values("mean_yield")
    .head(10)
)
low_yield.to_csv(f"{OUTPUT_DIR}/low_yield_zones.csv", index=False)

# ------------------------------------------
# 8. AUTOCORRELATION SPATIALE (MORAN I)
# ------------------------------------------
gdf_sn = gdf_sn.dropna(subset=["mean_yield"])
w = weights.Queen.from_dataframe(gdf_sn)
w.transform = "r"

# Moran global
moran = Moran(gdf_sn["mean_yield"], w)

with open(f"{OUTPUT_DIR}/moran_results.txt", "w") as f:
    f.write(f"Moran I : {moran.I}\n")
    f.write(f"P-value  : {moran.p_sim}\n")

# ------------------------------------------
# 9. CLUSTERS SPATIAUX (LISA)
# ------------------------------------------
lisa = Moran_Local(gdf_sn["mean_yield"], w)
gdf_sn["lisa_cluster"] = lisa.q

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
gdf_sn.plot(
    column="lisa_cluster",
    categorical=True,
    legend=True,
    ax=ax
)
ax.set_title("Clusters spatiaux des rendements (LISA) – Sénégal")
ax.axis("off")
plt.savefig(f"{OUTPUT_DIR}/senegal_lisa_clusters.png", dpi=300)
plt.close()

# ------------------------------------------
# 10. PLUVIAL VS IRRIGUE
# ------------------------------------------
system_yield = (
    df.groupby("crop_production_system")["yield"]
      .mean()
)
system_yield.to_csv(f"{OUTPUT_DIR}/yield_by_system.csv")

# ------------------------------------------
# 11. EVOLUTION TEMPORELLE
# ------------------------------------------
yield_time = (
    df.groupby("harvest_year")["yield"]
      .mean()
      .reset_index()
)

plt.figure(figsize=(8,5))
plt.plot(yield_time["harvest_year"], yield_time["yield"])
plt.xlabel("Année")
plt.ylabel("Rendement moyen (t/ha)")
plt.title("Evolution du rendement agricole au Sénégal")
plt.savefig(f"{OUTPUT_DIR}/yield_trend_senegal.png", dpi=300)
plt.close()

# ------------------------------------------
# 12. AREA vs YIELD
# ------------------------------------------
corr = df[["area", "yield"]].corr().iloc[0,1]

with open(f"{OUTPUT_DIR}/area_yield_correlation.txt", "w") as f:
    f.write(f"Correlation area-yield : {corr}\n")


C:\Users\HP\AppData\Local\Temp\ipykernel_26652\3586036178.py:88: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = weights.Queen.from_dataframe(gdf_sn)
C:\Users\HP\anaconda3\Lib\site-packages\libpysal\weights\contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 5 disconnected components.
 There are 4 islands with ids: 1, 13, 16, 37.
  W.__init__(self, neighbors, ids=ids, **kw)
C:\Users\HP\anaconda3\Lib\site-packages\esda\moran.py:1354: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


('WARNING: ', 1, ' is an island (no neighbors)')
('WARNING: ', 13, ' is an island (no neighbors)')
('WARNING: ', 16, ' is an island (no neighbors)')
('WARNING: ', 37, ' is an island (no neighbors)')
